In [1]:
%run -n cnn.py

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)
/home/Devansh/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
/home/Devansh/anaconda2/lib/python2.7/site-packages/theano/tensor/signal/downsample.py:6: UserWarning: downsample module has been moved to the theano.tensor.signal.pool module.
  "downsample module has been moved to the theano.tensor.signal.pool module.")


In [2]:
import sklearn.metrics
import pandas as pd
import csv

In [3]:
def get_data(dataset, train):
    print 'Loading', dataset
    # load in hickle dataset containing cell label dataset
    d = hickle.load('{}/{}.hkl'.format(DATA_DIR, dataset))

    GROUPS = {
        'NORMAL': 0,
        'Echinocyte': 1,
        'Dacrocyte': 2,
        'Schistocyte': 3,
        'Elliptocyte': 4,
        'Acanthocyte': 5,
        'Target cell': 6,
        'Stomatocyte': 7,
        'Spherocyte': 8,
        'Overlap': 9
    }
    LABELS = {v:k for k, v in GROUPS.items()}
    
    labels = []
    images = []
    pk = []
    
    # z = pk_value
    for y, x, z in zip(d['y'], d['X'], d['pk']):
        labels.append(GROUPS[y])
        images.append(x)
        pk.append(z)
    labels = np.array(labels)
    images = np.array(images)
    pk = np.array(pk)

    
    sss = sklearn.cross_validation.StratifiedShuffleSplit(
        labels,
        n_iter=1,
        test_size=0.2,
        random_state=RANDOM_SEED,
        )
    
    # create train set
    if train:
        ix, _ = tuple(sss)[0]
    # create validation set
    else:
        _, ix = tuple(sss)[0]

    labels = labels[ix]
    images = images[ix]
    pk = pk[ix]
    
    images = images / 255. - 0.5
    images = images.transpose(0, 3, 1, 2)
    
    for i in range(len(LABELS)):
        print '{} - {}: {}'.format(i, LABELS[i], (labels == i).sum())
        
    return images, labels

In [4]:
dataset = 'September_1_total_non_overlap'

In [5]:
output_layer, samplers = setup(dataset)

Layer 0 type: <class 'lasagne.layers.input.InputLayer'> output: (32, 3, 70, 70)
Layer 1 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 32, 68, 68)
Layer 2 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 32, 66, 66)
Layer 3 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 32, 64, 64)
Layer 4 type: <class 'lasagne.layers.dnn.MaxPool2DDNNLayer'> output: (32, 32, 31, 31)
Layer 5 type: <class 'lasagne.layers.noise.DropoutLayer'> output: (32, 32, 31, 31)
Layer 6 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 64, 29, 29)
Layer 7 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 64, 27, 27)
Layer 8 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 64, 25, 25)
Layer 9 type: <class 'lasagne.layers.dnn.MaxPool2DDNNLayer'> output: (32, 64, 12, 12)
Layer 10 type: <class 'lasagne.layers.noise.DropoutLayer'> output: (32, 64, 12, 12)
Layer 11 type: <class 'lasagne.layers.dnn.Conv2DDNNLayer'> output: (32, 128, 10, 10

In [6]:
f_pred = get_predictor(output_layer)

In [7]:
# change this
model = './output/models/september_1 epoch 175.pickle' 

In [8]:
print 'Dataset stats:'
X, y = get_data(dataset, train=False)

print '\nTesting model "{}"'.format(model.split('/')[-1].split(' ')[0])
_ = load_parameters(output_layer, model)
p = f_pred(X.astype('float32'))

print '\nAccuracy: ', (p.argmax(1) == y).mean()
print '\nConfusion matrix:\n', sklearn.metrics.confusion_matrix(y, p.argmax(1))
print '\nSummary classification metrics:\n', sklearn.metrics.classification_report(y, p.argmax(1))

Dataset stats:
Loading September_1_total_non_overlap
0 - NORMAL: 203
1 - Echinocyte: 63
2 - Dacrocyte: 18
3 - Schistocyte: 152
4 - Elliptocyte: 18
5 - Acanthocyte: 33
6 - Target cell: 145
7 - Stomatocyte: 22
8 - Spherocyte: 48
9 - Overlap: 46

Testing model "september_1"
Loaded network parameters from ./output/models/september_1 epoch 175.pickle

Accuracy:  0.975935828877

Confusion matrix:
[[203   0   0   0   0   0   0   0   0   0]
 [  1  62   0   0   0   0   0   0   0   0]
 [  1   0  13   3   0   0   0   0   1   0]
 [  0   1   1 146   0   4   0   0   0   0]
 [  0   0   0   1  17   0   0   0   0   0]
 [  0   0   0   2   0  31   0   0   0   0]
 [  0   0   0   0   0   0 145   0   0   0]
 [  1   0   0   0   0   0   0  21   0   0]
 [  0   0   0   1   0   0   0   1  46   0]
 [  0   0   0   0   0   0   0   0   0  46]]

Summary classification metrics:
             precision    recall  f1-score   support

          0       0.99      1.00      0.99       203
          1       0.98      0.98   